### 기본 설정

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install accelerate
!pip install peft
!pip install evaluate
!pip install wandb
!pip install datasets

In [ ]:
# 수정 필요
DATA_SIZE = 0.3
USE_LORA = False

### 데이터셋

In [ ]:
import pandas as pd

train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataset/cls_train_data.csv')
valid_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataset/cls_validation_data.csv')

train_df.head()

,speaker,empathy,listener
0,"엄마, 아기가 태어나니까 내가 부모로서 해야 할 게 참 많은 것 같아요.",3,그렇지? 아기 키우는 게 여간 어려운 일이 아니야.
1,어제 평소보다도 격하게 막 온몸을 써가면서 울더라고요. 얼마나 당황했는지 몰라요.,5,배가 고파서 그랬던 것 아닐까? 아기들은 배가 고프면 몸부림을 친단다.
2,맞아요. 젖을 물려주니 금세 뚝 그쳤어요. 난 분명히 순했을 것 같은데.,5,말도 마. 네가 얼마나 까탈스러웠는데. 우리 손주가 내 딸을 빼다 박았네.
3,잠도 못 자고 우는 아이 달래랴 수유하랴 머리카락이 다 빠지는 줄 알았어요.,3,"네가 어른 노릇, 부모 노릇을 하느라 고생이 정말 많구나."
4,그러면서 엄마 생각이 많이 났어요. 엄마는 직장까지 다니면서 나를 키우느라 얼마나 ...,2,우리 딸이 엄마가 되더니 철이 들었네. 하지만 네가 웃어주면 세상 모든 시름이 사라...


In [ ]:
# 데이터 크기 줄이기 위함
train_df = train_df.sample(frac=DATA_SIZE, random_state=42)
valid_df = valid_df.sample(frac=DATA_SIZE, random_state=42)

In [ ]:
from transformers import PreTrainedTokenizerFast

Q_TKN = "<Q>"
A_TKN = "<A>"
BOS = '</s>'
EOS = '</s>'
UNK = '<unk>'
MASK = '<unused0>'
SENT = '<sent>'
PAD = '<pad>'

# KoGPT2 토크나이저 로드
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
            bos_token=BOS, eos_token=EOS, unk_token=UNK,
            pad_token=PAD, mask_token=MASK)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
from datasets import Dataset
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np

train_dataset = Dataset.from_pandas(train_df)
valid_dataset = Dataset.from_pandas(valid_df)

label_classes = ['1', '2', '3', '4', '5']
mlb = MultiLabelBinarizer(classes=label_classes)

def get_cls_input(examples):
    inputs = examples['text']
    labels = [label.split(',') for label in examples['empathy']]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")

    labels_binary = mlb.fit_transform(labels)
    labels = labels_binary.astype(np.float32).tolist()

    model_inputs["labels"] = labels
    return model_inputs

# 데이터셋 전처리
train_inputs = train_dataset.map(get_cls_input, batched=True)
valid_inputs = valid_dataset.map(get_cls_input, batched=True)

Map:   0%|          | 0/181802 [00:00<?, ? examples/s]

Map:   0%|          | 0/22554 [00:00<?, ? examples/s]

### 모델 학습

In [ ]:
import wandb

wandb.login()

wandb: Currently logged in as: nkim12 (nkim123). Use `wandb login --relogin` to force relogin


True

In [ ]:
# method
sweep_config = {
    'method': 'random'
}

# hyperparameters
parameters_dict = {
    'lr_scheduler_type':{
        'values': ['linear', 'cosine', 'polynomial']
    },
    'learning_rate': {
        'distribution': 'log_uniform_values',
        'min': 1e-5,
        'max': 1e-3
    },
    'weight_decay': {
        'values': [0.1, 0.3, 0.5]
    },
    'train_batch_size': {
        'values': [8, 16, 32]
    },
    'eval_batch_size': {
        'values': [8, 16, 32]
    }
}

sweep_config['parameters'] = parameters_dict

In [ ]:
from peft import LoraConfig, TaskType, get_peft_model

peft_config = LoraConfig(
    task_type="SEQ_CLS",
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none")

In [ ]:
import evaluate

def compute_metrics(pred):
    logits, labels = pred
    predictions = (logits > 0).astype(float)  # 시그모이드 활성화 함수 적용 후 임계값 0.5 기준으로 이진
    labels = labels.flatten()
    predictions = predictions.flatten()

    # 정확도, F1 score 계산
    acc_metrics = evaluate.load("accuracy")
    accuracy = acc_metrics.compute(predictions=predictions, references=labels)

    f1_metrics = evaluate.load("f1")
    f1 = f1_metrics.compute(predictions=predictions, references=labels)

    return {"accuracy": accuracy, "f1_metrics": f1}

In [ ]:
from transformers import GPT2LMHeadModel, Trainer, TrainingArguments, EarlyStoppingCallback
from accelerate import Accelerator

def train():
  run = wandb.init()
  config = wandb.config

  # KoGPT2 모델 로드
  model = AutoModelForSequenceClassification.from_pretrained(
      "skt/kogpt2-base-v2",
      num_labels=5,
      problem_type="multi_label_classification"
  )
  if USE_LORA:
    # lora 적용
    accelerator = Accelerator() #데이터 병렬 처리 원활
    model = accelerator.prepare(model)
    model = get_peft_model(model, peft_config)

  # 학습 설정
  training_args = TrainingArguments(
      fp16=True,
      output_dir='./results',
      num_train_epochs=4,
      lr_scheduler_type=config.lr_scheduler_type,
      learning_rate=config.learning_rate,
      per_device_train_batch_size=config.train_batch_size,
      per_device_eval_batch_size=config.eval_batch_size,
      warmup_steps=10000,
      weight_decay=config.weight_decay,
      logging_dir='./logs',
      logging_steps=0.01,
      do_eval=True,
      eval_strategy="steps",
      eval_steps=0.1,
      remove_unused_columns=True,
  )

  # Trainer 설정
  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=train_inputs,
      eval_dataset=valid_inputs,
      compute_metrics=compute_metrics,
  )

  trainer.train()

  model_path = './kogpt2-classification'
  model.save_pretrained(model_path)
  tokenizer.save_pretrained(model_path)


In [ ]:
# 캐시 지우기
import torch, gc

gc.collect()
torch.cuda.empty_cache()

In [ ]:
sweep_id = wandb.sweep(sweep_config, entity='nkim123', project='minidlthon_kogpt2_classification')

wandb.agent(sweep_id, train, count=1)

wandb.finish()

Create sweep with ID: k9b3osx1
Sweep URL: https://wandb.ai/nkim123/minidlthon_kogpt2/sweeps/k9b3osx1


wandb: Agent Starting Run: c49d7baw with config:
wandb: 	eval_batch_size: 8
wandb: 	learning_rate: 1.6498371463751343e-05
wandb: 	lr_scheduler_type: cosine
wandb: 	train_batch_size: 16
wandb: 	weight_decay: 0.3


/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/layer.py:1091: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'weight_decay' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'lr_scheduler_type' was locked by 'sweep' (ignored update).


Step,Training Loss,Validation Loss
6,16.577500,16.858360
12,16.675300,16.858326
18,16.466500,16.858273
24,16.592200,16.858196
30,16.595600,16.858002
36,16.542400,16.857504
42,16.598900,16.857168
48,16.499900,16.856627
54,16.588300,16.856064


wandb: Adding directory to artifact (./kogpt2-chatbot)... Done. 0.0s


eval/loss,███▇▇▅▄▃▁
eval/runtime,▁▁▁▂▂▁▅█▆
eval/samples_per_second,███▆▇█▄▁▃
eval/steps_per_second,███▆▇█▄▁▃
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/grad_norm,▄▄▁▃▅▁▃▂▄▂▃▄▃▄▄▂▅█▆▂▃▄▄▂▂▄▄▅▃▆▂▆▃▃▄▃▄▅▄▄
train/learning_rate,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/loss,▆▆▄▄▅▅▇▆▇▁▄▆▄▆▅▅▆▇█▃▆▆▄▃▆▅▅▅▄▆▆█▄▄▆▅▅▅▄▇
eval/loss,16.85606
eval/runtime,1.2799
